In [2]:
import saspy
sas = saspy.SASsession(colorLOG=True)

Using SAS Config named: oda
SAS Connection established. Subprocess id is 17504



### data

In [7]:
sd = sas.sasdata("prdsale","sashelp")
sd.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,925.0,850.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-01-01
1,999.0,297.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-02-01
2,608.0,846.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-03-01
3,642.0,533.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-04-01
4,656.0,646.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,2.0,1993.0,1993-05-01


In [23]:
sas.submit(f"""
    data df1;
        set sashelp.prdsale;
        where actual = 925;
    run;
              
    data df2;
        set sashelp.prdsale;
        where actual = 999;
    run;
""")
sd1 = sas.sasdata("df1")
sd2 = sas.sasdata("df2")

In [24]:
sd1.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,925.0,850.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-01-01
1,925.0,345.0,CANADA,WEST,CONSUMER,OFFICE,TABLE,4.0,1994.0,1994-12-01


In [25]:
sd2.head()

,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
0,999.0,297.0,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1.0,1993.0,1993-02-01


### set statement


In [19]:
# Interleave observations by a var;
var = "predict"
sas.submitLST(f"""
    proc sort data = df1; by {var}; run;
    proc sort data = df2; by {var}; run;

    DATA interleave; 
        SET df1 df2; 
        BY {var}; 
    RUN;
              
    PROC PRINT DATA = interleave;
    RUN;
""")

Obs,ACTUAL,PREDICT,COUNTRY,REGION,DIVISION,PRODTYPE,PRODUCT,QUARTER,YEAR,MONTH
1,$999.00,$297.00,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1,1993,Feb
2,$925.00,$345.00,CANADA,WEST,CONSUMER,OFFICE,TABLE,4,1994,Dec
3,$925.00,$850.00,CANADA,EAST,EDUCATION,FURNITURE,SOFA,1,1993,Jan


### match merge 


In [26]:
# one to one
sas.submitLST(f"""
    data _a;
        set interleave;
        keep month actual;
    run;
              
    data _b;
        set interleave;
        keep month predict;
    run;
    
    proc sort data = _a; by month; run;
    proc sort data = _b; by month; run;

    data _c;
        merge _a _b;
        by month;
    run;

    proc print data = _c;
    run;
""")

Obs,ACTUAL,MONTH,PREDICT
1,$925.00,Jan,$850.00
2,$999.00,Feb,$297.00
3,$925.00,Dec,$345.00


In [29]:
# one to many
sas.submitLST(f"""
    data _a;
        set interleave;
        keep year actual;
    run;
              
    data _b;
        set interleave;
        keep year predict;
    run;
    
    proc sort data = _a nodupkey; by year; run;
    proc sort data = _b; by year; run;

    data _c;
        merge _a _b;
        by year;
    run;

    proc print data = _a;
    run;
    proc print data = _b;
    run;
    proc print data = _c;
    run;
""")

### merging back


In [37]:
sas.submitLST(f"""
proc means data = sashelp.prdsale noprint;
    by country;
    var predict;
    output out = _a mean = mean;
run;

proc sort data = _a; by country; run;
proc sort data = sashelp.prdsale out = prdsale; by country; run;

data _b;
    merge prdsale _a;
    by country;
    keep actual predict country mean;
run;

proc sort data = _b; by actual; run;

proc print data = _b (obs = 5);
run;

""")

Obs,ACTUAL,PREDICT,COUNTRY,mean
1,$3.00,$405.00,CANADA,$485.46
2,$3.00,$938.00,GERMANY,$482.40
3,$4.00,$975.00,U.S.A.,$503.59
4,$5.00,$425.00,CANADA,$485.46
5,$6.00,$214.00,CANADA,$485.46


In [38]:
print(f"{sas.sasdata("prdsale","sashelp").obs()=}")
print(f"{sas.sasdata("_b","work").obs()=}")

sas.sasdata("prdsale","sashelp").obs()=1440
sas.sasdata("_b","work").obs()=1440


### grand total


### update with transactions


### output statement


### options


### proc transpose


### automatic variables
